In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

In [ ]:
yelp = pd.read_json('yelp_academic_dataset_review.json')


In [ ]:
yelp.shape()

In [ ]:
yelp.head()
yelp.info()

In [ ]:
# describe
yelp.describe()

In [ ]:
yelp['text length'] = yelp['text'].apply(len)
yelp.head()

In [ ]:
# Dataset mapping
g = sns.FacetGrid(data=yelp, col='stars')
g.map(plt.hist, 'text length', bins=50)

In [ ]:
# Stars plotting for text length 
sns.boxplot(x='stars', y='text length', data=yelp)

In [ ]:
stars = yelp.groupby('stars').mean()
stars.corr()

In [ ]:
# Visualize correlations
sns.heatmap(data=stars.corr(), annot=True)
